In [ ]:
from binance import Client
import pandas as pd
import numpy as np

In [ ]:
client=Client()
btc=client.get_historical_klines('BTCUSDT', client.KLINE_INTERVAL_5MINUTE, '25 Nov, 2023')

In [ ]:
btc[0]

In [ ]:
len(btc)

In [ ]:
type(btc)

In [ ]:
btc_df=pd.DataFrame(btc, columns=['open_timestamp', 'open', 'high', 'low', 'close',
                                  'volume', 'close_timestamp', 'quote_asset_volume', 'n_trades',
                                  'taker_buy_base_asset_vol', 'taker_buy_quote_asset_vol', 'ignore'])

In [ ]:
btc_df.head(5)

In [ ]:
btc_df.drop(columns=['quote_asset_volume', 'taker_buy_base_asset_vol', 'taker_buy_quote_asset_vol', 'ignore'], inplace=True)

In [ ]:
btc_df.head(5)

In [ ]:
for i in ['open', 'close', 'low', 'high', 'volume']:
    btc_df[i]=pd.to_numeric(btc_df[i])

In [ ]:
btc_df.head(5)

In [ ]:
btc_df['ema_10']=btc_df['close'].ewm(span=10, min_periods=10).mean()
btc_df['ema_30']=btc_df['close'].ewm(span=20, min_periods=20).mean()
btc_df['ema_50']=btc_df['close'].ewm(span=50, min_periods=50).mean()
btc_df['ema_100']=btc_df['close'].ewm(span=100, min_periods=100).mean()
btc_df['ema_200']=btc_df['close'].ewm(span=200, min_periods=200).mean()

In [ ]:
def add_ema(df, *emas):
    for ema in emas:
        df[f'ema_{ema}']=df['close'].ewm(span=ema, min_periods=ema).mean()

add_ema(btc_df, 10, 30, 50, 100, 200)

In [ ]:
btc_df.head(200)

In [ ]:
btc_df.iloc[200]

In [ ]:
len(btc_df)

In [ ]:
def rsi(df: pd.DataFrame, period: int=14):
    price_diff=df['close'].diff()
    gain=(price_diff.where(price_diff>0, 0)).rolling(window=period).mean()
    loss=(-price_diff.where(price_diff<0, 0)).rolling(window=period).mean()

    rs=gain/loss
    df['rsi']=100-(100/(1+rs))

rsi(btc_df, period=14)

In [ ]:
btc_df.head(50)

In [ ]:
def macd(df: pd.DataFrame):
    short_ema=df['close'].ewm(span=12, adjust=False).mean()
    long_ema=df['close'].ewm(span=26, adjust=False).mean()

    df['macd_line']=short_ema-long_ema
    df['signal_line']=df['macd_line'].ewm(span=9, adjust=False).mean()

macd(btc_df)

In [ ]:
btc_df.head(50)

In [ ]:
def bollinger_bands(df: pd.DataFrame, period: int=20):
    copy=df.copy()
    copy['sma']=df['close'].rolling(window=period).mean()
    df['upper_band']=copy['sma']+(df['close'].rolling(window=period).std()*2)
    df['lower_band']=copy['sma']-(df['close'].rolling(window=period).std()*2)

bollinger_bands(btc_df)

In [ ]:
btc_df.head(50)

In [ ]:
def stochastic_oscillator(df: pd.DataFrame, k_period=14, d_period=3):
    high=df['high'].rolling(window=k_period).max()
    low=df['low'].rolling(window=k_period).min()
    df['k']=100*((df['close']-low)/(high-low))
    df['d']=df['k'].rolling(window=d_period).mean()

stochastic_oscillator(btc_df)

In [ ]:
btc_df.head(50)

In [ ]:
def wwma(values: pd.Series, alpha):
    return values.ewm(alpha=1/alpha, min_periods=alpha, adjust=False).mean()
    
def atr(df: pd.DataFrame, period=14):
    data=df.copy()
    high=data['high']
    low=data['low']
    close=data['close']
    data['tr_0']=abs(high-low)
    data['tr_1']=abs(high-close.shift())
    data['tr_2']=abs(low-close.shift())
    tr=data[['tr_0', 'tr_1', 'tr_2']].max(axis=1)
    df['atr']=wwma(tr, period)

atr(btc_df)

In [ ]:
btc_df.head(30)

In [ ]:
def chaikin_money_flow(df: pd.DataFrame, period=20):
    data=btc_df.copy()
    data['money_flow_multiplier']=((data['close']-data['low'])-(data['high']-data['close']))/(data['high']-data['low'])
    data['money_flow_volume']=data['money_flow_multiplier']*data['volume']
    df['cmf']=data['money_flow_volume'].rolling(window=period).sum()/data['volume'].rolling(window=period).sum()

chaikin_money_flow(btc_df)

In [ ]:
btc_df.head(30)

In [ ]:
# hour of the day
btc_df['open_time']=pd.to_datetime(btc_df['open_timestamp'], unit='ms')
btc_df['close_time']=pd.to_datetime(btc_df['close_timestamp'], unit='ms')

In [ ]:
btc_df.head()

In [ ]:
btc_df['hour_open']=btc_df['open_time'].dt.hour
btc_df['hour_close']=btc_df['close_time'].dt.hour

In [ ]:
btc_df.iloc[801:820]

In [ ]:
def set_value(x):
    if x==5 or x==6:
        return True
    else:
        return False
    
btc_df['weekend']=btc_df['open_time'].dt.weekday.apply(set_value)

In [ ]:
btc_df.head()

In [ ]:
len(btc_df[btc_df['weekend']==False]), len(btc_df[btc_df['weekend']==True])

In [ ]:
btc_df['open_time'].dt.weekday

In [ ]:
btc_df['day']=btc_df['open_time'].dt.weekday

In [ ]:
btc_df['day'][:1600]

In [ ]:
btc_df['day'].unique()

In [ ]:
import math
days=[]
for i in btc_df['day']:
    days.append(i)

In [ ]:
len(days)

In [ ]:
days[1600]

In [ ]:
sin_days=[]
cos_days=[]
for i in days:
    sin_days.append(math.sin(2*math.pi*i/7))
    cos_days.append(math.cos(2*math.pi*i/7))

In [ ]:
sin_days[1600]

In [ ]:
cos_days[1600]

In [ ]:
sin_days_df=pd.DataFrame(sin_days, columns=['sin_days'])
cos_days_df=pd.DataFrame(cos_days, columns=['cos_days'])

In [ ]:
sin_days_df.head()

In [ ]:
cos_days_df.head()

In [ ]:
len(sin_days_df), len(cos_days_df)

In [ ]:
btc_df['sin_days']=sin_days_df['sin_days']
btc_df['cos_days']=cos_days_df['cos_days']

In [ ]:
btc_df.iloc[1600]

In [ ]:
hours=[]
for i in btc_df['hour_open']:
    hours.append(i)

In [ ]:
len(hours)

In [ ]:
btc_df['hour_open'].unique()

In [ ]:
sin_hours=[]
cos_hours=[]
for i in hours:
    sin_hours.append(math.sin(2*math.pi*i/24))
    cos_hours.append(math.cos(2*math.pi*i/24))

In [ ]:
sin_hours_df=pd.DataFrame(sin_hours, columns=['sin_hours'])
cos_hours_df=pd.DataFrame(cos_hours, columns=['cos_hours'])

In [ ]:
sin_hours_df.iloc[701:720]

In [ ]:
cos_hours_df[701:720]

In [ ]:
btc_df['sin_hours']=sin_hours_df['sin_hours']
btc_df['cos_hours']=cos_hours_df['cos_hours']

In [ ]:
len(btc_df)

In [ ]:
btc_df.head()

In [ ]:
btc_df.drop(columns=['open_time', 'close_time', 'day',], inplace=True)

In [ ]:
btc_df.drop(columns=['hour_open', 'hour_close'], inplace=True)

In [ ]:
btc_df